# Limpieza de datos del SNIIM (Sistema Nacional de información e Integración de Mercados)

Una vez descargadas todas las tablas del SNIIM en formato CSV, se unifican en un sólo dataframe que contenga toda la información necesaria.

Por ejemplo, un dataframe que unifique a todas las frutas y hortalizas reportadas en el periodo del **01/01/2020 al 31/12/2023** tendrá **más de 2 millones** de observaciones, con periodicidad diara.

#### Importar librerías

In [1]:
import pandas as pd
import os
import re
import numpy as np

#### Creación del dataframe con todos los archivos CSV descargados

In [2]:
# Se crea un dataframe que contiene la información de todos los archivos CSV
path = "./raw_data/"
archivos = next(os.walk(path))[2]
#print("Archivos = ", archivos)

lista_df = []

for i in range(len(archivos)):
    df_temp = pd.read_csv(path + archivos[i], thousands=',') # uno de los archivos utiliza commas en sus números

    # Se remueve el row que sólo contiene la categoría del producto "Frutas","Hortalizas"
    df_temp = df_temp.drop([0])

    # Se crea columna nueva
    nombre_producto = re.search(r"Precio_de_(.+)_\d{4}-\d{4}.csv", archivos[i]).group(1).replace("_", " ")
    df_temp['Producto'] = nombre_producto

    lista_df.append(df_temp)

# Se concatenan todos en un sólo dataframe
df_sniim_fyh = pd.concat(lista_df, ignore_index=True)

# Se renombran columnas
df_sniim_fyh.columns= df_sniim_fyh.columns.str.lower()
df_sniim_fyh.rename(columns={"presentación":"presentacion", "precio mín":"precio_min", "precio max":"precio_max", "precio frec":"precio_frec","obs.": "observaciones"}, inplace=True)

# Se cambia el tipo de dato de variable
df_sniim_fyh['fecha'] = pd.to_datetime(df_sniim_fyh['fecha'], dayfirst=True, format='%d/%m/%Y')

df_sniim_fyh


,fecha,presentacion,origen,destino,precio_min,precio_max,precio_frec,observaciones,producto
0,2020-01-02,Caja de 20 kg.,San Luis Potosí,Aguascalientes: Centro Comercial Agropecuario ...,190.0,210.0,200.0,NaN,Chayote
1,2020-01-03,Caja de 20 kg.,San Luis Potosí,Aguascalientes: Centro Comercial Agropecuario ...,190.0,210.0,200.0,NaN,Chayote
2,2020-01-06,Caja de 20 kg.,San Luis Potosí,Aguascalientes: Centro Comercial Agropecuario ...,190.0,210.0,200.0,NaN,Chayote
3,2020-01-07,Caja de 20 kg.,San Luis Potosí,Aguascalientes: Centro Comercial Agropecuario ...,190.0,210.0,200.0,NaN,Chayote
4,2020-01-08,Caja de 20 kg.,San Luis Potosí,Aguascalientes: Centro Comercial Agropecuario ...,200.0,220.0,210.0,NaN,Chayote
...,...,...,...,...,...,...,...,...,...
2086678,2021-01-14,Kilogramo,Nuevo León,"Nuevo León: Mercado de Abasto ""Estrella"" de Sa...",7.0,8.0,7.5,NaN,Naranja Albérchiga mediana
2086679,2021-01-15,Kilogramo,Nuevo León,"Nuevo León: Mercado de Abasto ""Estrella"" de Sa...",7.0,10.0,9.0,NaN,Naranja Albérchiga mediana
2086680,2021-02-15,Kilogramo,Nuevo León,"Nuevo León: Mercado de Abasto ""Estrella"" de Sa...",7.0,9.0,8.0,NaN,Naranja Albérchiga mediana
2086681,2021-02-16,Kilogramo,Nuevo León,"Nuevo León: Mercado de Abasto ""Estrella"" de Sa...",7.0,9.0,8.0,NaN,Naranja Albérchiga mediana


#### Análisis básico del dataframe y algunas características

La variable "**Observaciones**" es la única que presenta valores perdidos, más del 95% de las observaciones en esta variable son valores perdidos. Se entiende que es una variable opcional que sólo indica algún comentario.

In [3]:
df_sniim_fyh.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2086683 entries, 0 to 2086682
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   fecha          2086683 non-null  datetime64[ns]
 1   presentacion   2086683 non-null  object        
 2   origen         2086683 non-null  object        
 3   destino        2086683 non-null  object        
 4   precio_min     2086683 non-null  float64       
 5   precio_max     2086683 non-null  float64       
 6   precio_frec    2086683 non-null  float64       
 7   observaciones  90870 non-null    object        
 8   producto       2086683 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 143.3+ MB


Algunas estadísticas descriptivas sobre las variables numéricas:

In [4]:
df_sniim_fyh.describe().apply(lambda s: s.apply('{0:.2f}'.format))

,precio_min,precio_max,precio_frec
count,2086683.00,2086683.00,2086683.00
mean,184.94,205.86,194.53
std,259.09,286.28,269.41
min,1.00,1.00,0.00
25%,19.00,21.00,20.00
50%,65.00,72.00,70.00
75%,255.00,290.00,275.00
max,11000.00,6650.00,4200.00


Explorando la variable categórica **producto** se puede ver el número total de productos en el dataframe (más de 200 productos), y el número de observaciones con las que cuenta cada producto (por ejemplo: para el producto *"Tomate Saladette"* se encuentan más de 40 mil observaciones.

In [5]:
df_sniim_fyh['producto'].value_counts()

Jitomate Saladette          44917
Tomate Saladette            44917
Manzana Golden Delicious    37192
Papa Alpha                  36522
Manzana Red Delicious       35416
                            ...  
Melón Cantaloupe # 18          20
Mango Mangloba                 10
Pistache                        5
Melón Cantaloupe # 17           2
Melón Gotas de miel             1
Name: producto, Length: 221, dtype: int64

No se encuentran observaciones duplicadas:

In [6]:
df_sniim_fyh[df_sniim_fyh.duplicated()]

,fecha,presentacion,origen,destino,precio_min,precio_max,precio_frec,observaciones,producto
